In [1]:
import numpy as np
import pandas as pd
import json
import math
import matplotlib.pyplot as plt
import seaborn as sns
from keplergl import KeplerGl
import requests
from datetime import datetime
import networkx as nx
import osmnx as ox
from shapely.geometry import LineString
from shapely import to_geojson
import os

%matplotlib inline

In [2]:
def create_mds_data(df, G):
    df.sort_values('timestamp', inplace=True, ignore_index=True)

    df['lat'] = df['lat'].apply(lambda x: round(x,3))
    df['lon'] = df['lon'].apply(lambda x: round(x,3))
    reduced_df = df.drop_duplicates(['bike_id','lat', 'lon'])
    point_counts = reduced_df['bike_id'].value_counts()
    ids = point_counts[point_counts > 1].keys()

    reduced_df = reduced_df[reduced_df['bike_id'].isin(ids)].reset_index(drop=True)
    reduced_df['timestamp'] = pd.to_datetime(reduced_df['timestamp'], format='%Y-%m-%dT%H:%M:%S.%fZ')


    dates, bike_ids, edges, lengths, global_edges = [], [], [], [], []

    for id in ids:
        trip = reduced_df[reduced_df['bike_id'] == id].reset_index(drop=True)
        for i in range(1,len(trip.index)):
            edges_i = []
            length = 0
            start = ox.nearest_nodes(G, trip['lon'][i-1], trip['lat'][i-1])
            end = ox.nearest_nodes(G, trip['lon'][i], trip['lat'][i])
            path = ox.shortest_path(G, start, end)
            try:
                for j in range(1,len(path)):
                    length += G.get_edge_data(path[j-1],path[j])[0]['length']
                    edges_i.append((path[j-1],path[j]))
                    global_edges.append((path[j-1],path[j]))
                dates.append(trip['timestamp'][0].date())
                bike_ids.append(id)
                edges.append(edges_i)
                lengths.append(length)
            except:
                pass

    trip_df = pd.DataFrame({'date':dates, 'bike_id':bike_ids, 'trip_edges':edges, 'trip_length':lengths})

    

    return trip_df, global_edges

In [3]:
def get_name_shape(G:nx.classes.multidigraph.MultiDiGraph) -> pd.DataFrame:
    nodes = ox.graph_to_gdfs(G, edges=False)
    nodes_dict = nodes.to_dict()

    G_edges = list(G.edges())
    G_shapes = []
    G_names = []
    for edge in G_edges:
        try:
            name_i = G.get_edge_data(edge[0],edge[1])[0]['name']
            G_names.append(name_i)
        except:
            G_names.append(None)
        try:
            shape_i = G.get_edge_data(edge[0],edge[1])[0]['geometry']
            G_shapes.append(shape_i)
        except:
            start = (nodes_dict['x'][edge[0]],nodes_dict['y'][edge[0]])
            end = (nodes_dict['x'][edge[1]],nodes_dict['y'][edge[1]])
            G_shapes.append(LineString([start,end]))

    #G_edges = [str(edge) for edge in G_edges]
    #G_shapes = [str(edge) for edge in G_shapes]
    #G_names = [str(edge) for edge in G_names]

    name_shape_df = pd.DataFrame({'edge':G_edges, 'name':G_names, 'shape':G_shapes})
    #edge_name_dict = dict(zip(G_edges,G_names))
    #edge_shape_dict = dict(zip(G_edges,G_shapes))
    return name_shape_df

In [19]:
G = ox.graph_from_place('Prague, Czechia', network_type='drive')

In [5]:
name_shape_df = get_name_shape(G)

In [6]:
import sys
sys.path.append('../')

In [10]:
url = 'https://pilot.dksr.city/UrbanPulseData/historic/sensordata?eventTypeName=NextBikePragV1EventType&since=2024-09-01T00:00:00.000Z&until=2024-09-01T24:00:00.000Z'
text = requests.get(url, auth=(user, pw))

data = json.loads(text.text)
df = pd.json_normalize(data['sensordata'])

In [17]:
df.head()

,is_reserved,last_updated,vehicle_type_id,is_disabled,station_id,lon,bike_id,lat,timestamp,SID,_headers.eventType
0,False,1725148870,200,False,27582120,14.420880,meAPNpjQJgoojvMS3N5BbAgla350E4Zx,50.088020,2024-09-01T00:01:30.955Z,cefc35b9-c15d-4533-9170-6adc218c5bc3,NextBikePragV1EventType
1,False,1725148870,200,False,27582843,14.341610,AlNRdW7x4DqobJ8hVjYeEZ6VbQ1PG9KM,50.075320,2024-09-01T00:01:30.955Z,dc9e72be-584c-475b-bff4-9d9cfecc7a31,NextBikePragV1EventType
2,False,1725148870,245,False,409657556,14.604190,eZ34GQYqynO3yKBtKEGAjrnPA7OrpRlW,50.029495,2024-09-01T00:01:30.956Z,42c45569-a199-4ba2-ba75-8cc837eb0d26,NextBikePragV1EventType
3,False,1725148870,254,False,436331180,14.212440,jy1Nkd4GvnRzkd3HlOMXvVnmRAopVLW7,50.027039,2024-09-01T00:01:30.956Z,fc9cae73-49f1-4800-b6fc-6a9eff22b172,NextBikePragV1EventType
4,False,1725148870,200,False,27582981,14.648557,kQejW0KGXDNNeobuPVdJk2DLqYrax85P,50.000523,2024-09-01T00:01:30.956Z,c6f6ed6e-3fdb-4b56-b349-0511c174c8c8,NextBikePragV1EventType


In [30]:
trips, global_edges  = create_mds_data(df, G)

In [32]:
look_up = get_name_shape(G)

In [33]:
look_up

,edge,name,shape
0,"(172512, 300107261)",Štefánikova,"LINESTRING (14.4041771 50.0769181, 14.4042167 ..."
1,"(172512, 25675011)",Zubatého,"LINESTRING (14.4041771 50.0769181, 14.4041447 ..."
2,"(172512, 109901443)",Štefánikova,"LINESTRING (14.4041771 50.0769181, 14.4041685 ..."
3,"(172513, 172514)",náměstí Kinských,"LINESTRING (14.4041524 50.0787628, 14.404246 5..."
4,"(172513, 21713478)",náměstí Kinských,"LINESTRING (14.4041524 50.0787628, 14.4041841 ..."
...,...,...,...
49803,"(12175411981, 12175411980)",None,"LINESTRING (14.6281783 50.1164855, 14.6280315 ..."
49804,"(12179138075, 445771212)",K Náhonu,"LINESTRING (14.5299303 50.1025487, 14.5298901 ..."
49805,"(12179191875, 281154767)",Hany Maškové,"LINESTRING (14.5227626 50.1064796, 14.5227705 ..."
49806,"(12179191875, 281154798)",Nepelova,"LINESTRING (14.5227626 50.1064796, 14.5228843 ..."
